In [2]:
import cv2
import time
import requests
import re
import numpy as np
from typing import Optional
from ultralytics import YOLO
from sort.sort import Sort
from util import get_car, reader

BASE_URL     = "http://visaonestbackend-env.eba-25r2yw9h.us-east-2.elasticbeanstalk.com/cars"
DETECT_URL   = f"{BASE_URL}/detect"
RELEASE_PATH = "{plate}/release"  

CHAR_TO_DIGIT = {'O': '0', 'I': '1', 'J': '3', 'A': '4', 'G': '6', 'S': '5'}
DIGIT_TO_CHAR = {v: k for k, v in CHAR_TO_DIGIT.items()}

PLATE_REGEX = re.compile(r'^[A-Z]{3}[A-Z0-9]{4}$')

def normalize_plate(text: str) -> Optional[str]:
    cleaned = re.sub(r'[^A-Za-z0-9]', '', text).upper()
    if cleaned.startswith("BR"):
        cleaned = cleaned[2:]
    if len(cleaned) == 7 and PLATE_REGEX.fullmatch(cleaned):
        return cleaned
    return None

def extract_plate_text(thresh_img):
    for _, text, score in reader.readtext(thresh_img, detail=1):
        plate = normalize_plate(text)
        print(f" OCR raw = '{text}', normalizado = '{plate}', score = {score:.2f}")
        if plate:
            return plate
    return None

def is_registered(plate: str) -> bool:
    """Verifica se a placa já existe no backend (GET /cars/:plate)."""
    url = f"{BASE_URL}/{plate}"
    try:
        r = requests.get(url, timeout=5)
    except requests.RequestException as e:
        print(f"[ERRO] falha ao verificar registro da placa {plate}: {e}")
        return False
    if r.status_code == 200:
        print(f"[INFO] placa {plate} já cadastrada.")
        return True
    if r.status_code == 404:
        print(f"[INFO] placa {plate} não encontrada no cadastro.")
        return False
    print(f"[WARN] código inesperado ao verificar {plate}: {r.status_code}")
    return False

def register_plate(track_id: int, plate: str):
    """Envia deteccao ao backend (POST /cars/detect)."""
    payload = {"trackId": track_id, "plate": plate}
    print(f"[AÇÃO] registrando placa {plate} (trackId={track_id})…")
    try:
        r = requests.post(DETECT_URL, json=payload, timeout=5)
        r.raise_for_status()
        print(f"[SUCESSO] placa {plate} cadastrada ({r.status_code}).")
    except requests.RequestException as e:
        print(f"[ERRO] falha ao cadastrar {plate}: {e}")

def release_plate(plate: str) -> bool:
    """Solicita liberacao da placa (POST /cars/:plate/release)."""
    url = f"{BASE_URL}/{plate}/release"
    print(f"[AÇÃO] solicitando liberação da placa {plate}…")
    try:
        r = requests.post(url, timeout=5)
        r.raise_for_status()
    except requests.RequestException as e:
        print(f"[ERRO] falha na liberação de {plate}: {e}")
        return False

    resp_json = r.json()
    print(r.status_code, resp_json)

    allowed = resp_json.get("allowed", False)
    if allowed:
        print(f"[SUCESSO] placa {plate} liberada.")
        # crs teste
        print("carro liberado, ativando portao")
    else:
        print(f"[INFO] backend não autorizou liberação de {plate}.")

    return allowed


def preprocess_crop(crop):
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    return cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV, 35, 15
    )

def main():
    veh_model = YOLO("yolov8n.pt")
    lp_model  = YOLO("/home/messyas/ml/jetson/placas-model/models/runs/yolov8n_lp/weights/best.pt")
    tracker   = Sort()

    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)
    if not cap.isOpened():
        raise RuntimeError("Nao foi possivel abrir camera")

    DETECTION_INTERVAL = 5.0  # segundos
    last_detection = time.time() - DETECTION_INTERVAL

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("[ERRO] falha ao capturar frame.")
                break

            cv2.imshow("Webcam", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

            now = time.time()
            if now - last_detection >= DETECTION_INTERVAL:
                last_detection = now
                print(f"\n=== execução de detecção em {time.strftime('%H:%M:%S')} ===")

                # 1) Detecta veículos
                veh_results = veh_model(frame)[0]
                veh_boxes = [
                    b for b in veh_results.boxes.data.tolist()
                    if int(b[5]) in [2, 3, 5, 7]
                ]

                # 2) Atualiza tracker 
                if veh_boxes:
                    dets = np.array([
                        [x1, y1, x2, y2, score]
                        for x1, y1, x2, y2, score, _ in veh_boxes
                    ], dtype=float)
                    tracks = tracker.update(dets)
                else:
                    tracks = []

                # 3) Detecta placas 
                lp_results = lp_model(frame)[0]
                for box in lp_results.boxes.data.tolist():
                    x1, y1, x2, y2, score, _ = box
                    *_, track_id = get_car(box, tracks)
                    if track_id == -1:
                        continue

                    crop = frame[int(y1):int(y2), int(x1):int(x2)]
                    plate = extract_plate_text(preprocess_crop(crop))
                    if not plate:
                        print("[INFO] nenhuma placa valida detectada.")
                        continue

                    print(f"[PLACA] detectada: {plate}")

                    if not is_registered(plate):
                        register_plate(track_id, plate)
                    else:
                        release_plate(plate)

    except KeyboardInterrupt:
        print("Encerrado pelo usuario.")
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


=== execução de detecção em 22:12:17 ===

0: 480x640 (no detections), 4.2ms
Speed: 0.6ms preprocess, 4.2ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.0ms
Speed: 0.6ms preprocess, 4.0ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

=== execução de detecção em 22:12:22 ===

0: 480x640 1 keyboard, 5.7ms
Speed: 1.2ms preprocess, 5.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.6ms
Speed: 0.6ms preprocess, 4.6ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

=== execução de detecção em 22:12:27 ===

0: 480x640 (no detections), 4.5ms
Speed: 1.1ms preprocess, 4.5ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.4ms
Speed: 0.8ms preprocess, 4.4ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

=== execução de detecção em 22:12:32 ===

0: 480x640 (no detections), 4.7ms
Speed: 1.0ms pre